# Load Dataset


In [11]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import os
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import PIL
from tensorflow.keras import layers, Model
from tensorflow.keras.layers import Dense, Layer, Conv2D, MaxPool2D,GlobalAveragePooling2D,BatchNormalization
import time

from IPython import display

In [2]:
import urllib
categories = [line.rstrip(b'\n') for line in urllib.request.urlopen('https://raw.githubusercontent.com/googlecreativelab/quickdraw-dataset/master/categories.txt')]
print(categories[:10])
category = 'candle'

[b'aircraft carrier', b'airplane', b'alarm clock', b'ambulance', b'angel', b'animal migration', b'ant', b'anvil', b'apple', b'arm']


In [3]:
# Creates a folder to download the original drawings into.
# We chose to use the numpy format : 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

if not os.path.isdir('npy_files'):
    os.mkdir('npy_files')
    
url = f'https://storage.googleapis.com/quickdraw_dataset/full/numpy_bitmap/{category}.npy'  
urllib.request.urlretrieve(url, f'npy_files/{category}.npy')

images = np.load(f'npy_files/{category}.npy')
print(f'{len(images)} images to train on')

# You can limit the amount of images you use for training by setting :
train_images = images[:10000]
# You should also define a samller subset of the images for testing..
# TODO
test_images = images[10000:12000]
print(len(test_images))
print(len(train_images))
# Notice that this to numpy format contains 1x784 pixel vectors, with values going from 0 (white) to 255 (black). We reshape them later to 28x28 grids and normalize the pixel intensity to [-1, 1]

141545 images to train on
2000
10000


In [4]:
# define shuffle and batching sizes
batch_size = 32
shuffle_size = 1000
prefetch_size = 32

In [7]:
def preprocess(data):
    # change dtype to float32
    data = data.map(lambda img: tf.cast(img, tf.float32))
    # change value range from 0-255 to -1 - 1
    data = data.map(lambda img: (img/128)-1) 
    # change format to 28,28,1
    data = data.map(lambda img: tf.reshape(img, (28,28,1)))
    # do other prepocessing stuff
    data = data.shuffle(shuffle_size).batch(batch_size).prefetch(prefetch_size)
    return data




# Data Pipeline


In [6]:
# create datasets from tensor
train_ds = tf.data.Dataset.from_tensor_slices(train_images)
test_ds = tf.data.Dataset.from_tensor_slices(test_images)

train_ds = preprocess(train_ds)
test_ds = preprocess(test_ds)

#check data format real quick
# for x in test_ds:
#     print(x)

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-01-13 16:07:02.099529: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-01-13 16:07:02.099623: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Discriminator 

In [13]:
class Discriminator(Model):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv1 = Conv2D(32,kernel_size=2, padding='same')
        self.conv2 = Conv2D(32, kernel_size=3, padding='same')
        self.batch1 = BatchNormalization()

        self.conv3 = Conv2D(32, kernel_size=2, padding='same')
        self.conv4 = Conv2D(32, kernel_size=2, padding='same')
        self.batch2 = BatchNormalization()

        self.out = Dense(1, activation='sigmoid')

    @tf.function
    def call(self, input):
        x = self.conv1(input)
        x = self.conv2(x)
        x = self.batch1(x)
        x = self.conv3(input)
        x = self.conv4(x)
        x = self.batch2(x)
        x = self.out(x)
        return x 
